In [72]:
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data.dataloader import default_collate
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [73]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [74]:
"""
Download data from FashionMNIST
"""
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [75]:
"""
Define a simple neural network with 2 layers:
Layer 1: d_in:784, d_out:512 with relu activation
Layer 2: d_in:512, d_out:10 with no activation
"""
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512, bias=False),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [76]:
model = SimpleNN().to(device)
print(model)

SimpleNN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=False)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [77]:
learning_rate = 1e-3
batch_size = 64
epochs = 10
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [78]:
train_loader = DataLoader(training_data,batch_size=batch_size,collate_fn=lambda x:tuple(x_.to(device) for x_ in default_collate(x)))
test_loader = DataLoader(test_data,batch_size=batch_size,collate_fn=lambda x:tuple(x_.to(device) for x_ in default_collate(x)))

In [79]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X.float())
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X.float())
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [80]:
"""
Train the simple NN.
"""
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer)
    test_loop(test_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.314195  [    0/60000]
loss: 2.264490  [ 6400/60000]
loss: 2.168918  [12800/60000]
loss: 2.154444  [19200/60000]
loss: 2.100799  [25600/60000]
loss: 2.012174  [32000/60000]
loss: 2.025415  [38400/60000]
loss: 1.919454  [44800/60000]
loss: 1.900433  [51200/60000]
loss: 1.810286  [57600/60000]
Test Error: 
 Accuracy: 62.6%, Avg loss: 1.800888 

Epoch 2
-------------------------------
loss: 1.832915  [    0/60000]
loss: 1.800831  [ 6400/60000]
loss: 1.639667  [12800/60000]
loss: 1.697538  [19200/60000]
loss: 1.576061  [25600/60000]
loss: 1.528580  [32000/60000]
loss: 1.555407  [38400/60000]
loss: 1.451401  [44800/60000]
loss: 1.469964  [51200/60000]
loss: 1.350921  [57600/60000]
Test Error: 
 Accuracy: 64.9%, Avg loss: 1.379132 

Epoch 3
-------------------------------
loss: 1.435818  [    0/60000]
loss: 1.426373  [ 6400/60000]
loss: 1.240793  [12800/60000]
loss: 1.348863  [19200/60000]
loss: 1.223870  [25600/60000]
loss: 1.218792  [32000/600

In [81]:
"""
Use the degree 4 polynomial to approximate |x|
"""
pwr = np.array([1,2])
coefs = np.array([1.9303,-1.0655])
rng = np.random.default_rng()

In [82]:
"""
Define the random feature map as a pytorch module
"""
class RandomFeatureMap(nn.Module):
    def __init__(self, size_in, pwr, coefs):
        """
        Build a feature map with output dimension equals input dimension

        size_in: input dimension d
        pwr: powers in the polynomial
        coefs: coefficients of the polynomial
        """
        super().__init__()
        self.size_in, self.pwr, self.coefs = size_in, pwr, torch.tensor(coefs)
        self.max_pwr = pwr[-1]
        rng = np.random.default_rng()
        N = torch.tensor(rng.choice(self.pwr,size=(size_in,1)),dtype=torch.int32) # randomly generate (size_in) many N
        self.N = nn.Parameter(N,requires_grad=False)
        weights = [nn.Parameter(torch.tensor(rng.choice([-1.0,1.0], size=(self.size_in, 2*n.item()))), requires_grad=False) for n in self.N] # for each N, generate N random Rademacher vectors
        self.weights = nn.ParameterList(weights)
        signs = torch.tensor([torch.sign(self.coefs[n.item() - 1]) for n in self.N]) # store the signs
        self.signs = nn.Parameter(signs,requires_grad= False)
    def forward(self, x):
        """
        x -> a_N * n * \prod^N_{i=1} weights_i * x
        """
        return torch.stack([torch.sqrt(torch.abs(self.coefs[n.item() - 1]) * self.max_pwr) * torch.prod(torch.matmul(weight.T, x),dim=0) for weight, n in zip(self.weights, self.N)])

            

In [83]:
"""
Sanity check
"""
F = RandomFeatureMap(784,pwr,coefs).float()
x = torch.rand((784,64))
print(F(x).shape)

torch.Size([784, 64])


In [84]:
"""
Define the class of approximated neural networks
"""
class ApproxNN(nn.Module):
    def __init__(self, model):
        super(ApproxNN, self).__init__()
        self.Flatten = nn.Flatten() # flatten the input
        self.RandomFeatureMap = RandomFeatureMap(28*28,pwr,coefs).float() # Initialize the random feature map

        # Initialize two linear layers with weights from the trained neural network.
        self.Linear1 = nn.Linear(28*28,512, bias = False)
        self.Linear1.weight = nn.Parameter(torch.clone(model.get_parameter(target='linear_relu_stack.0.weight')))
        self.Linear2 = nn.Linear(512,10)
        self.Linear2.weight = nn.Parameter(torch.clone(model.get_parameter(target='linear_relu_stack.2.weight')))
        self.Linear2.bias = nn.Parameter(torch.clone(model.get_parameter(target='linear_relu_stack.2.bias')))
        
    def forward(self, x):
        """
        For the first layer:
        x -> 1/2(W.T * x + phi(W).T * S * phi(x))
        """
        x_flat = self.Flatten(x)

        x1 = self.Linear1(x_flat)

        x2 = torch.tensor(x_flat.T) # [d, n]
        x2_norm = torch.norm(x2,dim = 0) # [n]
        phi_x = self.RandomFeatureMap(nn.functional.normalize(x2, dim = 0)) * x2_norm # [d, n]

        W = self.Linear1.weight.T # [d, D]
        W_norm = torch.norm(W, dim = 0) # [D]
        phi_W = self.RandomFeatureMap(nn.functional.normalize(W, dim = 0)) * W_norm # [d, D]

        x = (x1.T + phi_W.T @ torch.diag(self.RandomFeatureMap.signs) @ phi_x)/2

        # second layer
        logits = self.Linear2(x.T)
        
        return logits

In [85]:
approx_model = ApproxNN(model).to(device)

In [86]:
test_loop(test_loader,approx_model,loss_fn)

C:\Users\Haolin\AppData\Local\Temp/ipykernel_34720/121419359.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x2 = torch.tensor(x_flat.T) # [d, n]


Test Error: 
 Accuracy: 43.7%, Avg loss: 219.803552 



In [87]:
optimizer = torch.optim.SGD(approx_model.parameters(), lr=learning_rate)

In [88]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, approx_model, loss_fn, optimizer)
    test_loop(test_loader, approx_model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


C:\Users\Haolin\AppData\Local\Temp/ipykernel_34720/121419359.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x2 = torch.tensor(x_flat.T) # [d, n]


loss: 251.175980  [    0/60000]
loss:     nan  [ 6400/60000]
loss:     nan  [12800/60000]
loss:     nan  [19200/60000]
loss:     nan  [25600/60000]
loss:     nan  [32000/60000]
loss:     nan  [38400/60000]
loss:     nan  [44800/60000]


KeyboardInterrupt: 